In [1]:
import pandas as pd
import numpy as np
import Levenshtein
import re
from  translate import Translator
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_score
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Reshape

translator = Translator(to_lang="en")


In [2]:
unique = pd.read_csv("../dataset/result_cleaning_data.csv")
data = pd.read_csv("../dataset/cleaned_data.csv")

print(unique.head())
print(data.head())

C:\Users\Rizal\AppData\Local\Temp\ipykernel_17664\1811373139.py:2: DtypeWarning: Columns (194,195,196,250,258,261,262,263,270,271,272,273,274,275,317,330,332,333,337,338,339,340,344,345,346,347,351,352,353,354,358,359,360,361,365,366,367,368,372,373,374,375,379,380,381,382,384,385,386,387,389,390,408,460,469,478,485,487,488,494,495,496,497,498,503,504,505,506,507,512,513,514,515,516,521,522,523,524,525,528,529,530,531,532,613,622,624,625,626,646,647,648,649,650,651,659,660,661,662,663,668,669,670,671,672,677,678,679,680,681,691,692,696,697,702,703,704,709,710,711,718,719,720,727,728,729,736,737,738,746,747,748,749,750,755,756,757,758,759,764,765,766,767,768,773,774,775,776,777,782,783,784,785,786,791,792,793,794,795,800,801,802,803,804,809,810,811,812,813,818,819,820,821,822,827,828,829,830,831,836,837,838,839,840,845,846,847,848,849,850,851,860,861,862,866,867,868,872,873,874,875,876,877,878,879,880,881,882,883,884,885,899,900,901,906,907,908,913,914,915,920,921,922,927,928,929,934,93

   Unnamed: 0                      0                  1           2  \
0           0      software engineer             intern     manager   
1           1       computer science            general  management   
2           2  boy scouts of america           sciences  technology   
3           3  boy scouts of america           sciences  technology   
4           4                   inc.  software engineer       owner   

                         3                        4                       5  \
0       research assistant                 director         project manager   
1  business administration                  finance  mechanical engineering   
2        purdue university  the university of texas           college board   
3        purdue university  the university of texas           college board   
4                      llc                  founder                director   

                        6                         7                         8  \
0                

**experiences/{j}/title**

In [3]:
data["headline"]

0                                         Illinois Mutual
1                         National Pork Producers Council
2               PhD Student at Carnegie Mellon University
3       Founder @ AlcinoMedia | Facebook/Meta & Google...
4         Forensics Biomechanical Analyst at VA Forensics
                              ...                        
8934                                  Cultural Strategist
8935           Engagement Manager at Rackspace Technology
8936    Community Manager/ Developer Advocate -Astra S...
8937                 Tech Support Agent at 343 Industries
8938                      Sales, Marketing, & Management.
Name: headline, Length: 8939, dtype: object

In [4]:
data = data["headline"]

data

0                                         Illinois Mutual
1                         National Pork Producers Council
2               PhD Student at Carnegie Mellon University
3       Founder @ AlcinoMedia | Facebook/Meta & Google...
4         Forensics Biomechanical Analyst at VA Forensics
                              ...                        
8934                                  Cultural Strategist
8935           Engagement Manager at Rackspace Technology
8936    Community Manager/ Developer Advocate -Astra S...
8937                 Tech Support Agent at 343 Industries
8938                      Sales, Marketing, & Management.
Name: headline, Length: 8939, dtype: object

In [5]:
colomns = unique.iloc[5].dropna()
colomns

Unnamed: 0                       5
0                software engineer
1                             inc.
2                          student
3                            owner
                      ...         
1332              team development
1333          compensation analyst
1334                           inl
1335                          ling
1336                game developer
Name: 5, Length: 1338, dtype: object

In [6]:
colomns = [str(col) for col in colomns.to_list()]
colomns

['5',
 'software engineer',
 'inc.',
 'student',
 'owner',
 'idate',
 'project manager',
 'llc',
 'director',
 'founder',
 'marketing',
 'senior software engineer',
 'ceo',
 'manager',
 'product manager',
 'president',
 'software developer',
 'vice president',
 'co-founder',
 'strategy',
 'entrepreneur',
 'data scientist',
 'operations',
 'mba c',
 'consultant',
 'analytics',
 'program manager',
 'senior manager',
 'analyst',
 'design',
 'designer',
 'associate',
 'br',
 'data analyst',
 'development',
 'account manager',
 'engineer',
 'software development engineer',
 'project management',
 'graphic designer',
 'finance',
 'writer',
 '--',
 'phd c',
 'inc',
 'graduate student',
 'editor',
 'partner',
 'mba',
 'management',
 'data science',
 'web developer',
 'business development',
 'business analyst',
 'executive director',
 'operations manager',
 'research assistant',
 'data',
 'innovation',
 'senior consultant',
 'sql',
 'researcher',
 'data analytics',
 'data engineer',
 'product'

Pencarian kedekatan dengan menggunakan cosine similarity\
Cosine Similarity mengukur kesamaan antara dua vektor dengan mengukur sudut kosinus di antara keduanya. Dengan menggunakan TF-IDF (Term Frequency-Inverse Document Frequency), Anda bisa mengonversi kalimat menjadi vektor yang merepresentasikan pentingnya kata-kata dalam kalimat tersebut.

In [7]:

def most_similarity(input_word, data):
    most_similar_word = None
    highest_similarity = -1

    for word in data:
        distance = Levenshtein.distance(input_word, word)
        similarity = 1 - (distance / max(len(input_word), len(word)))

        if similarity > highest_similarity:
            highest_similarity = similarity
            most_similar_word = word

    return most_similar_word

input_word = "kardi"

sample = ["Kardus", "Karpet", "Karikatur"]

output = most_similarity(input_word, sample)

print(output)  


Kardus


In [8]:
def extract_each_title(title_list):
    exp_titles = []
    parts = str(title_list).lower().replace("and", ",").replace("|", ",").replace("/", ",").replace("&", ",")\
        .replace(" – ", ",").replace(" - ", ",").split(",")
    
    parts = [part.strip() for part in parts]
    
    parts = [part.split(" at ")[0].strip() for part in parts]
    parts = [part.split(" on ")[0].strip() for part in parts]
    parts = [part for part in parts if len(part) > 1]
    parts = [re.sub(r"[()]", "", part) for part in parts]
    parts = [translator.translate(part) for part in parts]
    
    exp_titles.extend(parts)
    # print(exp_titles)
    return exp_titles

title_each_people = [extract_each_title(data.drop(columns="full_name").iloc[i]) for i in range(len(data)) if str(i) != "nan"]
title_each_people


[['illinois mutual'],
 ['national pork producers council'],
 ['phd student'],
 ['founder @ alcinomedia',
  'facebook',
  'meta',
  'google ads advertiser for attorneys'],
 ['forensics biomechanical analyst'],
 ['risk analyst chez coface'],
 ['actor', 'o resort'],
 ['sociology student'],
 ['td process integration engineer'],
 ['cnc programmer'],
 ['mobile xamarin', 'ios native developer'],
 ['mechanical engineer maintenance service advisor'],
 ['social media manager'],
 ['vp of product'],
 ['data engineer', 'analytics'],
 ['quality engineer ii'],
 ['account representative', 'inc.'],
 ['management executive',
  'award winning crm',
  'cs instructor',
  '10+ dir. ops',
  'entreprenuer'],
 ['student'],
 ['search engine optimization analyst'],
 ['ict',
  'business analysis',
  'project management',
  'data science researcher',
  'mba student'],
 ['regional key account manager presso eyes future s.r.l concessionario esclusivo bausch',
  'lomb surgical sicilia e calabria'],
 ['solutions archi

In [9]:
title_each_people = [" ".join(title) for title in title_each_people if len(title) != 0]
title_each_people

['illinois mutual',
 'national pork producers council',
 'phd student',
 'founder @ alcinomedia facebook meta google ads advertiser for attorneys',
 'forensics biomechanical analyst',
 'risk analyst chez coface',
 'actor o resort',
 'sociology student',
 'td process integration engineer',
 'cnc programmer',
 'mobile xamarin ios native developer',
 'mechanical engineer maintenance service advisor',
 'social media manager',
 'vp of product',
 'data engineer analytics',
 'quality engineer ii',
 'account representative inc.',
 'management executive award winning crm cs instructor 10+ dir. ops entreprenuer',
 'student',
 'search engine optimization analyst',
 'ict business analysis project management data science researcher mba student',
 'regional key account manager presso eyes future s.r.l concessionario esclusivo bausch lomb surgical sicilia e calabria',
 'solutions architect',
 'associate scientist',
 'mba in finance',
 'hse advisor quarries ajcq',
 'graduate reseach teaching assistant

In [10]:
np.zeros(len(colomns))

array([0., 0., 0., ..., 0., 0., 0.])

In [11]:
index_col = {}

for i in range(len(colomns)):
    index_col[colomns[i]] = i

index_col

{'5': 0,
 'software engineer': 1,
 'inc.': 2,
 'student': 3,
 'owner': 4,
 'idate': 5,
 'project manager': 6,
 'llc': 7,
 'director': 8,
 'founder': 9,
 'marketing': 10,
 'senior software engineer': 11,
 'ceo': 12,
 'manager': 13,
 'product manager': 14,
 'president': 15,
 'software developer': 16,
 'vice president': 17,
 'co-founder': 18,
 'strategy': 19,
 'entrepreneur': 20,
 'data scientist': 21,
 'operations': 22,
 'mba c': 23,
 'consultant': 24,
 'analytics': 25,
 'program manager': 26,
 'senior manager': 27,
 'analyst': 28,
 'design': 29,
 'designer': 30,
 'associate': 31,
 'br': 32,
 'data analyst': 33,
 'development': 34,
 'account manager': 35,
 'engineer': 36,
 'software development engineer': 37,
 'project management': 38,
 'graphic designer': 39,
 'finance': 40,
 'writer': 41,
 '--': 42,
 'phd c': 43,
 'inc': 44,
 'graduate student': 45,
 'editor': 46,
 'partner': 47,
 'mba': 48,
 'management': 49,
 'data science': 50,
 'web developer': 51,
 'business development': 52,
 'bu

In [12]:
def sentences_count(titles):
    frequency = [0 for i in range(len(colomns))]

    normalize_data = [most_similarity(title, colomns) for title in titles]

    for title in normalize_data:
        frequency[index_col[title]] += 1
    return frequency

# title_frequencies = [sentences_count(title) for title in title_each_people ]
# title_frequencies
    

In [13]:
title_each_people[0]

'illinois mutual'

In [14]:
title_each_people = [most_similarity(title, colomns) for title in title_each_people]

In [15]:
# Use TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(title_each_people)

# Convert the TF-IDF matrix to a DataFrame
new_data = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
new_data


,2022,2023,3d,aa,academic,account,accountant,accounting,accounts,acquisition,...,web,welder,wellness,whitney,work,worker,working,worship,writer,youth
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.524639,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8911,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8912,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8913,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8914,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Parameter VAE
input_dim = new_data.shape[1]
intermediate_dim = 64
latent_dim = 2

# Encoder
inputs = Input(shape=(input_dim,), dtype='float32')
h = Dense(intermediate_dim, activation='relu')(inputs)

z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(lambda x: sampling(x), output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder part of the VAE
decoder_h = Dense(intermediate_dim, activation='relu')  # Intermediate layer
decoder_mean = Dense(input_dim, activation='sigmoid')  # Output layer matching input_dim

# Applying the layers
h_decoded = decoder_h(z)  # Apply the intermediate layer to the latent space 'z'
x_decoded_mean = decoder_mean(h_decoded)  # Apply the output layer to the result of the intermediate layer
x_decoded_mean = tf.expand_dims(x_decoded_mean, axis=0)


# VAE model
vae = Model(inputs, x_decoded_mean)

# Calculate reconstruction loss
reconstruction_loss = tf.reduce_mean(tf.square(inputs - x_decoded_mean))
reconstruction_loss *= input_dim  # Scale by the number of elements

# Compute the KL divergence
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

# Combine the losses
vae_loss = K.mean(reconstruction_loss + kl_loss)

# Add the loss to the model and compile
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(new_data, epochs=100, batch_size=2, shuffle=True)




Epoch 1/100
4458/4458 [==============================] - 33s 7ms/step - loss: 3.9127
Epoch 2/100
4458/4458 [==============================] - 31s 7ms/step - loss: 0.9749
Epoch 3/100
4458/4458 [==============================] - 41s 9ms/step - loss: 0.9729
Epoch 4/100
4458/4458 [==============================] - 39s 9ms/step - loss: 0.9722
Epoch 5/100
4458/4458 [==============================] - 55s 12ms/step - loss: 0.9720
Epoch 6/100
4458/4458 [==============================] - 35s 8ms/step - loss: 0.9720
Epoch 7/100
4458/4458 [==============================] - 48s 11ms/step - loss: 0.9719
Epoch 8/100
4458/4458 [==============================] - 62s 14ms/step - loss: 0.9718
Epoch 9/100
4458/4458 [==============================] - 67s 15ms/step - loss: 0.9718
Epoch 10/100
4458/4458 [==============================] - 53s 12ms/step - loss: 0.9717
Epoch 11/100
4458/4458 [==============================] - 38s 9ms/step - loss: 0.9716
Epoch 12/100
4458/4458 [==============================]

In [17]:
new_data.shape

(8916, 588)

In [18]:
# Define encoder model to extract latent space
encoder = Model(inputs, z_mean)

# Ekstraksi fitur kompresi dari latent space
compressed_features = encoder.predict(new_data.to_numpy())
print(compressed_features)


279/279 [==============================] - 1s 4ms/step
[[-0.00039161 -0.00102576]
 [-0.00039161 -0.00102576]
 [-0.00039161 -0.00102576]
 ...
 [-0.00039161 -0.00102576]
 [-0.00039161 -0.00102576]
 [-0.00039161 -0.00102576]]


In [19]:
input_dim = new_data.shape[1]  # Sesuaikan dengan data Anda
intermediate_dim = 500  # Ukuran layer intermediate, dapat disesuaikan
latent_dim = compressed_features.shape[1]  # Dimensi latent space
n_clusters = 4  # Jumlah cluster

# Clustering layer
class ClusteringLayer(Layer):
    def __init__(self, n_clusters, **kwargs):
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters

    def build(self, input_shape):
        self.input_dim = input_shape[1]
        self.clusters = self.add_weight(shape=(self.n_clusters, self.input_dim), initializer='glorot_uniform', name='clusters')

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2))
        q **= (self.input_dim + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

# Define DEC model
inputs = Input(shape=(input_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)

z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# DEC specific layers
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(z)
dec = Model(inputs, clustering_layer)

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(compressed_features)
dec.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# Custom training loop
optimizer = SGD(0.01, 0.9)

# Convert inputs to TensorFlow variable
X_tf = tf.convert_to_tensor(new_data.to_numpy(), dtype=tf.float32)

epochs = 100
batch_size = 2

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        q = dec(X_tf)
        p = tf.pow(q, 2) / tf.reduce_sum(q, axis=0)
        p /= tf.reduce_sum(p, axis=1, keepdims=True)
        loss = tf.reduce_mean(tf.keras.losses.KLD(p, q))
    grads = tape.gradient(loss, dec.trainable_weights)
    optimizer.apply_gradients(zip(grads, dec.trainable_weights))
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Assign clusters to data points
q = dec.predict(new_data.to_numpy())
y_pred_final = q.argmax(1)

# Menampilkan hasil clustering
new_data['Cluster'] = y_pred_final
print(new_data)

c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Epoch 0, Loss: 0.0
Epoch 10, Loss: 0.0
Epoch 20, Loss: 0.0
Epoch 30, Loss: 0.0
Epoch 40, Loss: 0.0
Epoch 50, Loss: 0.0
Epoch 60, Loss: 0.0
Epoch 70, Loss: 0.0
Epoch 80, Loss: 0.0
Epoch 90, Loss: 0.0
279/279 [==============================] - 2s 6ms/step
      2022  2023   3d   aa  academic   account  accountant  accounting  \
0      0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
1      0.0   0.0  0.0  0.0       0.0  0.524639         0.0         0.0   
2      0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
3      0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
4      0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
...    ...   ...  ...  ...       ...       ...         ...         ...   
8911   0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
8912   0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
8913   0.0   0.0  0.0  0.0       0.0  0.000000         0.0         0.0   
8914  

In [20]:
print(compressed_features.shape)  # Pastikan ini sesuai dengan latent_dim yang diharapkan


(8916, 2)


In [21]:
new_data["Cluster"].value_counts()

Cluster
0    8916
Name: count, dtype: int64

In [22]:
# Menghitung Silhouette Score
score = silhouette_score(compressed_features, y_pred_final)
print(f'Silhouette Score: {score}')


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
def find_best_k(data, max_k=16):
    inertia = []
    silhouette_scores = []

    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, n_init=20)
        kmeans.fit(data)
        
        # Append inertia (within-cluster sum of squares)
        inertia.append(kmeans.inertia_)
        
        # Compute silhouette score
        score = silhouette_score(data, kmeans.labels_)
        silhouette_scores.append(score)

    # Plotting the Elbow Method
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range(2, max_k + 1), inertia, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    

    # Plotting the Silhouette Score
    plt.subplot(1, 2, 2)
    plt.plot(range(2, max_k + 1), silhouette_scores, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score for Optimal k')

    plt.show()

    # Best k based on silhouette score
    best_k = silhouette_scores.index(max(silhouette_scores)) + 2
    print(f'Best k based on Silhouette Score: {best_k}')

    return best_k

# Assuming compressed_features is your data after feature extraction
best_k = find_best_k(compressed_features, max_k=16)

c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
input_dim = new_data.shape[1]  # Sesuaikan dengan data Anda
intermediate_dim = 500  # Ukuran layer intermediate, dapat disesuaikan
latent_dim = compressed_features.shape[1]  # Dimensi latent space
n_clusters = 4  # Jumlah cluster


# Define DEC model
inputs = Input(shape=(input_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)

z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# DEC specific layers
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(z)
dec = Model(inputs, clustering_layer)

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(compressed_features)
dec.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# Custom training loop
optimizer = SGD(0.01, 0.9)

# Convert inputs to TensorFlow variable
X_tf = tf.convert_to_tensor(new_data.to_numpy(), dtype=tf.float32)

epochs = 100
batch_size = 2

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        q = dec(X_tf)
        p = tf.pow(q, 2) / tf.reduce_sum(q, axis=0)
        p /= tf.reduce_sum(p, axis=1, keepdims=True)
        loss = tf.reduce_mean(tf.keras.losses.KLD(p, q))
    grads = tape.gradient(loss, dec.trainable_weights)
    optimizer.apply_gradients(zip(grads, dec.trainable_weights))
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Assign clusters to data points
q = dec.predict(new_data.to_numpy())
y_pred_final = q.argmax(1)

# Menampilkan hasil clustering
new_data['Cluster'] = y_pred_final
print(new_data)

Epoch 0, Loss: 1.3162383538656286e-06
Epoch 10, Loss: 1.1272303481746349e-06
Epoch 20, Loss: 6.966286605347705e-07
Epoch 30, Loss: 4.462404490368499e-07
Epoch 40, Loss: 2.757622326043929e-07
Epoch 50, Loss: 1.4945729276405473e-07
Epoch 60, Loss: 8.002753304481303e-08
Epoch 70, Loss: 4.609021431178917e-08
Epoch 80, Loss: 2.7314282391444067e-08
Epoch 90, Loss: 1.6808883174235234e-08
32/32 [==============================] - 1s 7ms/step
     boy scouts of america  sciences  technology  purdue university  \
0                        0         0           0                  0   
1                        0         0           0                  0   
2                        0         0           0                  0   
3                        0         0           0                  0   
4                        0         0           0                  0   
..                     ...       ...         ...                ...   
991                      0         0           0                  

In [ ]:
vae.save('../model/headline/vae_model.h5')
dec.save('../model/headline/dec_model.h5')

c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
